In [6]:
%pip install openai-whisper

  Using cached openai_whisper-20240930-py3-none-any.whl
  Using cached numba-0.61.0-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached torch-2.6.0-cp311-cp311-win_amd64.whl.metadata (28 kB)
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.7 kB 682.7 kB/s eta 0:00:01
     -------------------- ----------------- 30.7/57.7 kB 330.3 kB/s eta 0:00:01
     -------------------------- ----------- 41.0/57.7 kB 281.8 kB/s eta 0:00:01
     -------------------------------------- 57.7/57.7 kB 379.5 kB/s eta 0:00:00
  Using cached more_itertools-10.6.0-py3-none-any.whl.metadata (37 kB)
  Using cached tiktoken-0.9.0-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Using cached llvmlite-0.44.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     ---------------------------------------- 41.5/41.5 kB ? eta 0:00:00
  Using cached sympy-1.13.1-py3-none-any.whl.


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install numpy==2.0.0

     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.9 kB ? eta -:--:--
     ------------ ------------------------- 20.5/60.9 kB 320.0 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.9 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 60.9/60.9 kB 359.3 kB/s eta 0:00:00
   ---------------------------------------- 0.0/16.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.5 MB 1.9 MB/s eta 0:00:09
   ---------------------------------------- 0.0/16.5 MB 1.9 MB/s eta 0:00:09
   ---------------------------------------- 0.1/16.5 MB 544.7 kB/s eta 0:00:31
   ---------------------------------------- 0.1/16.5 MB 853.3 kB/s eta 0:00:20
   ---------------------------------------- 0.2/16.5 MB 952.6 kB/s eta 0:00:18
    --------------------------------------- 0.2/16.5 MB 860.2 kB/s eta 0:00:19
    --------------------------------------- 0.3/16.5 MB 947.5 kB/s eta 0:00


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install sounddevice


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
Using cached sounddevice-0.5.1-py3-none-win_amd64.whl (363 kB)
   ---------------------------------------- 0.0/181.4 kB ? eta -:--:--
   -- ------------------------------------- 10.2/181.4 kB ? eta -:--:--
   ------ -------------------------------- 30.7/181.4 kB 435.7 kB/s eta 0:00:01
   -------- ------------------------------ 41.0/181.4 kB 393.8 kB/s eta 0:00:01
   ------------------- ------------------- 92.2/181.4 kB 581.0 kB/s eta 0:00:01
   ------------------------------ ------- 143.4/181.4 kB 708.1 kB/s eta 0:00:01
   -------------------------------------- 181.4/181.4 kB 781.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/117.6 kB ? eta -:--:--
   -------------------- ------------------- 61.4/117.6 kB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 117.6/117.6 kB 2.3 MB/s eta 0:00:00


In [7]:
import sounddevice as sd
import numpy as np
import whisper
import queue
import torch

# Параметры записи
SAMPLERATE = 16000  # Частота дискретизации (Whisper требует 16kHz)
CHANNELS = 1        # Один канал (моно)
CHUNK_DURATION = 5  # Длина каждого куска в секундах
OVERLAP = 2         # Перекрытие между кусками в секундах
MODEL_SIZE = "medium"  # Можно "tiny", "base", "small", "medium", "large"
LANGUAGE = "ru"  # Язык распознавания

# Очередь для хранения аудиоданных
audio_queue = queue.Queue()

# Загрузка модели
print("Загрузка модели Whisper...")
device = "cuda" if torch.cuda.is_available() else "cpu"
model = whisper.load_model(MODEL_SIZE, device=device)
print("Модель загружена!")

Загрузка модели Whisper...
Модель загружена!


In [8]:
# Функция обработки аудио в фоне
def callback(indata, frames, time, status):
    """Функция обратного вызова, получает аудиоданные"""
    if status:
        print(status)
    # Добавляем новый кусок в очередь
    audio_queue.put(indata.copy())

# Запускаем потоковую запись
with sd.InputStream(samplerate=SAMPLERATE, channels=CHANNELS, dtype='float32', callback=callback, blocksize=int(SAMPLERATE * (CHUNK_DURATION - OVERLAP))):
    print("Началась запись. Нажмите Ctrl+C для остановки.")
    
    # Кольцевой буфер для хранения аудиоданных
    ring_buffer = np.zeros(int(SAMPLERATE * CHUNK_DURATION), dtype=np.float32)
    
    try:
        while True:
            # Получаем новый кусок
            new_chunk = audio_queue.get()
            new_chunk = new_chunk[:, 0]  # Убираем лишнее измерение (канал)
            
            # Обновляем кольцевой буфер (сдвигаем влево, добавляем новый кусок справа)
            ring_buffer[:-len(new_chunk)] = ring_buffer[len(new_chunk):]
            ring_buffer[-len(new_chunk):] = new_chunk

            # Передаем в Whisper (Whisper сам обработает тишину)
            print("Обработка аудио...")
            result = model.transcribe(ring_buffer, language=LANGUAGE)
            print("📝 Распознанный текст:", result["text"])

    except KeyboardInterrupt:
        print("\nОстановка программы...")


Началась запись. Нажмите Ctrl+C для остановки.
Обработка аудио...


c:\Users\1\CU\mts_hack\mts-hakaton\.venv\Lib\site-packages\whisper\transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст:  СПОКОЙНАЯ МУЗЫКА
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст:  Так, пошла какая-то...
Обработка аудио...
📝 Распознанный текст:  пошла какая-то обработка аудио
Обработка аудио...
📝 Распознанный текст:  то где это работает вообще
Обработка аудио...
📝 Распознанный текст:  где это работает вообще.
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...
📝 Распознанный текст: 
Обработка аудио...

Остановка программы...
